In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import itertools

from upcycle.plotting.credible_regions import get_arm, draw_arm_comparison
from pathlib import Path

sns.set(style='whitegrid', font_scale=1.5)

In [ ]:
def aggregate_data(arms, keys):
    records = []
    for s_ratio in synth_ratios:
        for n_teach in num_teachers:
            metrics = dict(num_synth=s_ratio*50000, s_ratio=s_ratio, n_teach=n_teach)
            for key in keys:
                arm_tuple = get_arm(exp_dir, arms[f'{n_teach}-->1 ({s_ratio})'], 'student_train_metrics', 'epoch', key)
                metrics.update({
                    f'{key}_mean': arm_tuple[1][-1],
                    f'{key}_lb': arm_tuple[2][-1],
                    f'{key}_ub': arm_tuple[3][-1],
                })
            records.append(metrics)
    df = pd.DataFrame(records)
    return df

### Static Synthetic Augmentation

In [ ]:
exp_dir = '../data/experiments/image_classification'
dataset = 'cifar100'
version = 'v0.0.10'
teacher_depth = 56
student_depth = 56
synth_ratios = [0.0, 0.25, 0.5, 0.75, 1.0]
num_teachers = [5]
alpha = 0.0
num_classes = dict(cifar10=10, cifar100=100)

arms = {}
for s_ratio, n_teach in itertools.product(synth_ratios, num_teachers):
    arms[f'{n_teach}-->1 ({s_ratio})'] = f'synth_aug_{s_ratio}_{dataset}_ts_soft_cross_ent_alpha_{alpha}_{version}/'\
                                         f'preresnet{teacher_depth}_{n_teach}-preresnet{student_depth}_1'

In [ ]:
fig = plt.figure(figsize=(6, 5))
keys = ['test_ts_agree']

ax_2 = fig.add_subplot(1, 1, 1)
df = aggregate_data(arms, keys)

ax_2.plot(df.num_synth, df.test_ts_agree_mean, linewidth=2, color='blue')
ax_2.fill_between(df.num_synth, df.test_ts_agree_lb, df.test_ts_agree_ub, alpha=0.25, color='blue')
ax_2.scatter(df.num_synth, df.test_ts_agree_mean, color='blue')
ax_2.set_xlabel('# Synthetic Examples')
ax_2.set_ylabel('T/S Test Agreement')

# fig.suptitle(f'ResNet{teacher_depth}-3 --> ResNet{student_depth}, {dataset.upper()}')
plt.tight_layout()
# plt.savefig(f'figures/image_classification/synth_aug_ablation_{dataset}_test_agree.pdf')

In [ ]:
# for npz
saved_arrays = dict(
    num_synth = df.num_synth.values,
    test_ts_agree_mean = df.test_ts_agree_mean.values,
    test_ts_agree_lb = df.test_ts_agree_lb.values,
    test_ts_agree_ub = df.test_ts_agree_ub.values,
)

In [ ]:
fig = plt.figure(figsize=(6, 5))
keys = ['test_acc', 'teacher_test_acc']

ax_2 = fig.add_subplot(1, 1, 1)
df = aggregate_data(arms, keys)

ax_2.plot(df.num_synth, df.test_acc_mean, linewidth=2, color='blue', label='student')
ax_2.fill_between(df.num_synth, df.test_acc_lb, df.test_acc_ub, alpha=0.25, color='blue')
ax_2.scatter(df.num_synth, df.test_acc_mean, color='blue')
xlim = plt.xlim()
ax_2.set_xlim(xlim)
ax_2.set_ylim((70, 77))
ax_2.hlines(df.teacher_test_acc_mean.iloc[0], *xlim, linestyle='--', color='green', label='teacher')
ax_2.set_xlabel('# Synthetic Examples')
ax_2.set_ylabel('Test Accuracy')

# fig.suptitle(f'ResNet{teacher_depth}-3 --> ResNet{student_depth}, {dataset.upper()}')
plt.tight_layout()
plt.legend()
# plt.savefig(f'figures/image_classification/synth_aug_ablation_{dataset}_test_acc.pdf')

In [ ]:
saved_arrays.update(dict(
    test_acc_mean = df.test_acc_mean.values,
    test_acc_lb = df.test_acc_lb.values,
    test_acc_ub = df.test_acc_ub.values,
    teacher_test_acc_mean=df.teacher_test_acc_mean.values,
))

np.savez('num_synth_ablation.npz', **saved_arrays)

In [ ]:
saved_arrays = np.load('num_synth_ablation.npz')
list(saved_arrays.keys())

In [ ]:
fig = plt.figure(figsize=(6, 5))
keys = ['train_acc', 'teacher_train_acc']

ax_2 = fig.add_subplot(1, 1, 1)
df = aggregate_data(arms, keys)

ax_2.plot(df.num_synth, df.train_acc_mean, linewidth=2, color='blue', label='student')
ax_2.fill_between(df.num_synth, df.train_acc_lb, df.train_acc_ub, alpha=0.25, color='blue')
ax_2.scatter(df.num_synth, df.train_acc_mean, color='blue')
xlim = plt.xlim()
ax_2.set_xlim(xlim)
ax_2.hlines(df.teacher_train_acc_mean.iloc[0], *xlim, linestyle='--', color='green', label='teacher')
ax_2.set_xlabel('# Synthetic Examples')
ax_2.set_ylabel('Train Accuracy')

# fig.suptitle(f'ResNet{teacher_depth}-3 --> ResNet{student_depth}, {dataset.upper()}')
plt.tight_layout()
# plt.legend()
# plt.savefig(f'figures/image_classification/synth_aug_ablation_{dataset}_train_acc.pdf')

In [ ]:
fig = plt.figure(figsize=(6, 5))
keys = ['train_loss']

ax_2 = fig.add_subplot(1, 1, 1)
df = aggregate_data(arms, keys)

ax_2.plot(df.num_synth, df.train_loss_mean, linewidth=2, color='blue', label='student')
ax_2.fill_between(df.num_synth, df.train_loss_lb, df.train_loss_ub, alpha=0.25, color='blue')
ax_2.scatter(df.num_synth, df.train_loss_mean, color='blue')
ax_2.set_xlabel('# Synthetic Examples')
ax_2.set_ylabel('Train Loss')

# fig.suptitle(f'ResNet{teacher_depth}-3 --> ResNet{student_depth}, {dataset.upper()}')
plt.tight_layout()
# plt.legend()
# plt.savefig(f'figures/image_classification/synth_aug_ablation_{dataset}_train_acc.pdf')

### Online Synthetic Augmentation

In [ ]:
exp_dir = '../data/experiments/image_classification'
dataset = 'cifar100'
version = 'v0.0.10'
teacher_depth = 56
student_depth = 56
synth_ratios = [0.2, 0.4, 0.6, 0.8]
num_teachers = [5]
alpha = 0.0
temp = 4.0
num_classes = dict(cifar10=10, cifar100=100)

arms = {}
for s_ratio, n_teach in itertools.product(synth_ratios, num_teachers):
    arms[f'{n_teach}-->1 ({s_ratio})'] = f'synthetic_augmentation/'\
                                         f'{dataset}_synth_ratio_{s_ratio}_alpha_{alpha}_temp_{temp}/'\
                                         f'preresnet{teacher_depth}_{n_teach}-preresnet{student_depth}_1'

In [ ]:
fig = plt.figure(figsize=(6, 5))
keys = ['test_acc', 'teacher_test_acc']

ax_2 = fig.add_subplot(1, 1, 1)
df = aggregate_data(arms, keys)

ax_2.plot(df.s_ratio, df.test_acc_mean, linewidth=2, color='blue', label='student')
ax_2.fill_between(df.s_ratio, df.test_acc_lb, df.test_acc_ub, alpha=0.25, color='blue')
ax_2.scatter(df.s_ratio, df.test_acc_mean, color='blue')
xlim = plt.xlim()
ax_2.set_xlim(xlim)
ax_2.set_ylim((71, 77))
ax_2.hlines(df.teacher_test_acc_mean.iloc[0], *xlim, linestyle='--', color='green', label='teacher')
ax_2.set_xlabel('Synthetic Ratio')
ax_2.set_ylabel('Test Accuracy')

# fig.suptitle(f'ResNet{teacher_depth}-3 --> ResNet{student_depth}, {dataset.upper()}')
plt.tight_layout()
plt.legend(loc='lower left')
# plt.savefig(f'figures/image_classification/synth_aug_ablation_{dataset}_test_acc.pdf')

In [ ]:
fig = plt.figure(figsize=(6, 5))
keys = ['test_ts_agree']

ax_2 = fig.add_subplot(1, 1, 1)
df = aggregate_data(arms, keys)

ax_2.plot(df.s_ratio, df.test_ts_agree_mean, linewidth=2, color='blue')
ax_2.fill_between(df.s_ratio, df.test_ts_agree_lb, df.test_ts_agree_ub, alpha=0.25, color='blue')
ax_2.scatter(df.s_ratio, df.test_ts_agree_mean, color='blue')
ax_2.set_xlabel('Synthetic Ratio')
ax_2.set_ylabel('T/S Test Agreement')

# fig.suptitle(f'ResNet{teacher_depth}-3 --> ResNet{student_depth}, {dataset.upper()}')
plt.tight_layout()
# plt.savefig(f'figures/image_classification/synth_aug_ablation_{dataset}_test_agree.pdf')

In [ ]:
keys = ['train_acc', 'teacher_train_acc']

fig = plt.figure(figsize=(6, 5))
ax_2 = fig.add_subplot(1, 1, 1)
df = aggregate_data(arms, keys)

ax_2.plot(df.s_ratio, df.train_acc_mean, linewidth=2, color='blue', label='student')
ax_2.fill_between(df.s_ratio, df.train_acc_lb, df.train_acc_ub, alpha=0.25, color='blue')
ax_2.scatter(df.s_ratio, df.train_acc_mean, color='blue')
xlim = plt.xlim()
ax_2.set_xlim(xlim)
ax_2.hlines(df.teacher_train_acc_mean.iloc[0], *xlim, linestyle='--', color='green', label='teacher')
ax_2.set_xlabel('Synthetic Ratio')
ax_2.set_ylabel('Train Accuracy')

# fig.suptitle(f'ResNet{teacher_depth}-3 --> ResNet{student_depth}, {dataset.upper()}')
plt.tight_layout()
# plt.legend()
# plt.savefig(f'figures/image_classification/synth_aug_ablation_{dataset}_train_acc.pdf')